In [ ]:
import numpy as np
from pr3_utils import *
from stereo import *

### load data

In [ ]:
dataset = "10"
time_stamp,features,linear_velocity,angular_velocity,k,b,imu_T_cam = \
    load_data(f"../data/{dataset}.npz")
T = np.load(f"../data/{dataset}_EKF_localization_T_mean.npy")

### using **all** features

In [ ]:
feature_type = "all"

seen_count = np.zeros(features.shape[1], dtype=np.uint)
M_init = np.zeros([features.shape[1], 3])

for t in range(time_stamp.shape[0]):
    present_mask = get_seeing_mask(features[t])
    first_time_mask = np.logical_and(seen_count==0, present_mask)

    PC_camera = features_to_PC(features[t, first_time_mask, :], k, b)
    PC_camera_hom = np.hstack([PC_camera, np.ones([PC_camera.shape[0],1])])
    PC_world_hom  = (T[t] @ imu_T_cam @ PC_camera_hom.T).T
    PC_world = PC_world_hom[:,:3]/PC_world_hom[:,[3]]

    M_init[first_time_mask] = PC_world
    seen_count[present_mask] += 1
    
    if np.all(seen_count):
        print(f"seen all features at {t} stamp")
        break
    
print(np.sum(seen_count>0), features.shape[1])

In [ ]:
fig, ax = visualize_trajectory("EKF_localization", T, show_ori=False)
ax.scatter(M_init[:, 0], M_init[:, 1], 
           s=0.1, c='C4',label="features")
ax.legend()
fig.savefig(f'../img/{dataset}_EKF_mapping_{feature_type}_init.png', dpi=300)
plt.show()

In [ ]:
np.save(f"../data/{dataset}_EKF_mapping_M_init_{feature_type}",M_init)

### using mid-range features

In [ ]:
feature_type = "selected"

seen_count = np.zeros(features.shape[1], dtype=np.uint)
M_init = np.zeros([features.shape[1], 3])

for t in range(time_stamp.shape[0]):
    present_mask = get_seeing_mask(features[t], d_min=8, d_max=35)
    first_time_mask = np.logical_and(seen_count==0, present_mask)

    PC_camera = features_to_PC(features[t, first_time_mask, :], k, b)
    PC_camera_hom = np.hstack([PC_camera, np.ones([PC_camera.shape[0],1])])
    PC_world_hom  = (T[t] @ imu_T_cam @ PC_camera_hom.T).T
    PC_world = PC_world_hom[:,:3]/PC_world_hom[:,[3]]

    M_init[first_time_mask] = PC_world
    seen_count[present_mask] += 1
    
    if np.all(seen_count):
        print(f"seen all features at {t} stamp")
        break
print(np.sum(seen_count>2), features.shape[1])

In [ ]:
fig, ax = visualize_trajectory("EKF_localization", T, show_ori=False)
ax.scatter(M_init[seen_count>2, 0], M_init[seen_count>2, 1], 
           s=0.1, c='C4',label="features")
ax.legend()
fig.savefig(f'../img/{dataset}_EKF_mapping_{feature_type}_init.png', dpi=300)
plt.show()

In [ ]:
np.save(f"../data/{dataset}_EKF_mapping_M_init_{feature_type}", M_init)
np.save(f"../data/{dataset}_EKF_mapping_M_mask_{feature_type}", seen_count>2)